# Day 7: Pandas Basics


Action Item: Wes McKinney, who created pandas, has a great video tutorial. Watch it here and follow along by checking out the GitHub of his code.

Project Milestone: Manipulate some randomly generated data in a pandas dataframe. Make sure you know how to select a specific column, how to set the dataframe index (and, in particular, how to make it a datetime index), and how to select ranges of rows by index.

In [1]:
import pandas as pd
import numpy as np

### Generate random data and put in DataFrame

In [23]:
data = {
    'User':['Tim', 'Bruno', 'Jason', 'Marry', 'Clara', 'Jade'], 
    'Age':[1, 3, 2, 2, 3, 1],
    'Language':['En', 'Sp', 'Pt', 'Dt', 'En', 'Sp'],
    'Score':np.random.randint(0,3,(1,6))[0]
}
df = pd.DataFrame(data)

In [24]:
print(df)

    User  Age Language  Score
0    Tim    1       En      0
1  Bruno    3       Sp      2
2  Jason    2       Pt      0
3  Marry    2       Dt      0
4  Clara    3       En      2
5   Jade    1       Sp      2


### Select a specific column: 

In [25]:
print(df['Language'])

0    En
1    Sp
2    Pt
3    Dt
4    En
5    Sp
Name: Language, dtype: object


### Set data frame index

To make it more interesting let's create another dataframe that tells us the time when a test was given, which we will then later use as the index after joining the two dataframes.

In [33]:
from randomtimestamp import randomtimestamp

# Generate random timestamps
timestamp = []
for i in range(10):
    timestamp.append(randomtimestamp('2019'))

# put names and timestamps together in dictionary
data = {
    'User':['Tim', 'Bruno', 'Jason', 'Marry', 'Clara', 'Jade', 'Laura', 'Claudette', 'Mark', 'Gregor'], 
    'Time':timestamp
}

# convert to DF
df_time = pd.DataFrame(data)

In [34]:
df_time

,User,Time
0,Tim,17-05-2020 16:21:24
1,Bruno,10-09-2019 12:28:34
2,Jason,09-06-2019 17:24:49
3,Marry,22-05-2020 10:30:18
4,Clara,31-03-2020 04:09:17
5,Jade,30-07-2019 00:05:56
6,Laura,28-12-2019 03:54:58
7,Claudette,15-07-2019 09:18:34
8,Mark,06-04-2020 09:21:57
9,Gregor,24-08-2019 14:30:29


Make sure Pandas treats 'Time' as a datetime object

In [70]:
df_time['Time'] = pd.to_datetime(df_time['Time'], unit='s')
df_time

,User,Time
0,Tim,2020-05-17 16:21:24
1,Bruno,2019-10-09 12:28:34
2,Jason,2019-09-06 17:24:49
3,Marry,2020-05-22 10:30:18
4,Clara,2020-03-31 04:09:17
5,Jade,2019-07-30 00:05:56
6,Laura,2019-12-28 03:54:58
7,Claudette,2019-07-15 09:18:34
8,Mark,2020-06-04 09:21:57
9,Gregor,2019-08-24 14:30:29


In [71]:
df_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   User    10 non-null     object        
 1   Time    10 non-null     datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 288.0+ bytes


We now have 4 additional names, which are not in the original data frame. If we want to keep these when joining the dataframes together we could do an outer join, which would result in NaNs for the missing cells:

In [72]:
df_outer = pd.merge(df, df_time, how='outer')
df_outer

,User,Age,Language,Score,Time
0,Tim,1.0,En,0.0,2020-05-17 16:21:24
1,Bruno,3.0,Sp,2.0,2019-10-09 12:28:34
2,Jason,2.0,Pt,0.0,2019-09-06 17:24:49
3,Marry,2.0,Dt,0.0,2020-05-22 10:30:18
4,Clara,3.0,En,2.0,2020-03-31 04:09:17
5,Jade,1.0,Sp,2.0,2019-07-30 00:05:56
6,Laura,NaN,NaN,NaN,2019-12-28 03:54:58
7,Claudette,NaN,NaN,NaN,2019-07-15 09:18:34
8,Mark,NaN,NaN,NaN,2020-06-04 09:21:57
9,Gregor,NaN,NaN,NaN,2019-08-24 14:30:29


Alternatively we can skip the rows in the second data frame for which no scores exist. To do so we use inner join.

In [73]:
df_inner = pd.merge(df, df_time, how='inner')
df_inner

,User,Age,Language,Score,Time
0,Tim,1,En,0,2020-05-17 16:21:24
1,Bruno,3,Sp,2,2019-10-09 12:28:34
2,Jason,2,Pt,0,2019-09-06 17:24:49
3,Marry,2,Dt,0,2020-05-22 10:30:18
4,Clara,3,En,2,2020-03-31 04:09:17
5,Jade,1,Sp,2,2019-07-30 00:05:56


It might be sensible to set the index as the time when the data was collected

In [74]:
df_inner.set_index('Time', inplace=True)
df_inner

,User,Age,Language,Score
Time,,,,
2020-05-17 16:21:24,Tim,1,En,0
2019-10-09 12:28:34,Bruno,3,Sp,2
2019-09-06 17:24:49,Jason,2,Pt,0
2020-05-22 10:30:18,Marry,2,Dt,0
2020-03-31 04:09:17,Clara,3,En,2
2019-07-30 00:05:56,Jade,1,Sp,2


In [75]:
df_inner.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6 entries, 2020-05-17 16:21:24 to 2019-07-30 00:05:56
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   User      6 non-null      object
 1   Age       6 non-null      int64 
 2   Language  6 non-null      object
 3   Score     6 non-null      int32 
dtypes: int32(1), int64(1), object(2)
memory usage: 216.0+ bytes


In [76]:
df_inner.index

DatetimeIndex(['2020-05-17 16:21:24', '2019-10-09 12:28:34',
               '2019-09-06 17:24:49', '2020-05-22 10:30:18',
               '2020-03-31 04:09:17', '2019-07-30 00:05:56'],
              dtype='datetime64[ns]', name='Time', freq=None)

Select only entries from 2019

In [78]:
df_inner.loc['2019']

,User,Age,Language,Score
Time,,,,
2019-10-09 12:28:34,Bruno,3,Sp,2
2019-09-06 17:24:49,Jason,2,Pt,0
2019-07-30 00:05:56,Jade,1,Sp,2


Select only entries taken after September 2019 and before October 2020:

In [80]:
df_inner.loc['2019-10':'2020-09']

,User,Age,Language,Score
Time,,,,
2020-05-17 16:21:24,Tim,1,En,0
2019-10-09 12:28:34,Bruno,3,Sp,2
2020-05-22 10:30:18,Marry,2,Dt,0
2020-03-31 04:09:17,Clara,3,En,2


Figure out the average score by Age

In [83]:
df_inner.groupby(by='Age')['Score'].mean()

Age
1    1
2    0
3    2
Name: Score, dtype: int32

That's it for now.